In [3]:
import csv

In [74]:
import os
import time
import sys
import pandas as pd
from selenium.webdriver import Firefox
from timeit import default_timer


def get_expand_buttons(browser):
    """Return a list of expand buttons to click on."""
    return browser.find_elements_by_xpath("//span[@title='expand']")

def expand_all_categories(browser, category, search_depth=3, save='csv', all_pages=True):
    """Expand all categories on the page."""
    start_time = default_timer()
    expand_buttons = get_expand_buttons(browser)
    time.sleep(3)
    depth = 0
    print('num expand buttons ', len(expand_buttons))
    while depth < search_depth:
        start = default_timer()
        for button in expand_buttons:
            if button.is_displayed():
                button.click()
        time.sleep(3)
        expand_buttons = get_expand_buttons(browser)
        end = default_timer()
        print(f'depth of {depth} took {str(round((end-start)/60, 2))} minutes to open')
        depth += 1
    if save == 'csv':
        start = default_timer()
        lst = []
        for a in browser.find_elements_by_xpath('.//a'):
            lst.append(a.get_attribute('href'))
        links = pd.DataFrame(lst)
        links = links.iloc[7:-32,:]
        if all_pages:
            links.to_csv(f'seed_data/all_pages_{category}_d{depth}_{len(links)}.csv', sep=',', encoding='utf-8', header=None, index=False)
        else:
            links.to_csv(f'seed_categories/cat_pages_{category}_d{depth}_{len(links)}.csv', sep=',', encoding='utf-8', header=None, index=False)
        end = default_timer()
        print(str(round((end-start)/60, 2)) + f' minutes to save to csv {len(links)} hrefs')
    if save == False:
        pass
    end_time = default_timer()
    print(str(round((end_time-start_time)/60, 2)) + f' minutes to finish search')


def get_links(category, search_depth=3, save='csv'):
    """get the links from wikipedia's hidden category tree finder"""
    browser = Firefox()
    time.sleep(0.5)
    browser.get(f'https://en.wikipedia.org/wiki/Special:CategoryTree?target={category}&mode=all&namespaces=&title=Special%3ACategoryTree')
    time.sleep(0.5)
    category = category.replace(' ','_')
    expand_all_categories(browser, category, search_depth, save='csv', all_pages=True)

def get_categories(category, search_depth=5, save='csv'):
    """gets the links from wikipedia's hidden category tree finder"""
    browser = Firefox()
    time.sleep(0.5)
    browser.get(f'https://en.wikipedia.org/wiki/Special:CategoryTree?target={category}&mode=categories&namespaces=&title=Special%3ACategoryTree')
    time.sleep(0.5)
    category = category.replace(' ','_')
    expand_all_categories(browser, category, search_depth, save='csv', all_pages=False)

def only_open(category, search_depth=5):
    browser = Firefox()
    time.sleep(0.5)
    browser.get(f'https://en.wikipedia.org/wiki/Special:CategoryTree?target={category}&mode=all&namespaces=&title=Special%3ACategoryTree')
    time.sleep(0.5)
    category = category.replace(' ','_')
    expand_all_categories(browser, category, search_depth, save=False, all_pages=False)



In [38]:
articles = []
with open('seed_data/all_pages_mathematics_d1_1456.csv', 'r') as fin:
    reader = csv.reader(fin)
    for row in reader:
        articles.append(','.join(row))

In [43]:
articles[:10]

['https://en.wikipedia.org/w/index.php?title=Special:CategoryTree&target=Abstraction&mode=20&hideprefix=20&showcount=1',
 'https://en.wikipedia.org/w/index.php?title=Special:CategoryTree&target=Categories_requiring_diffusion&mode=20&hideprefix=20&showcount=1',
 'https://en.wikipedia.org/w/index.php?title=Special:CategoryTree&target=Formal_sciences&mode=20&hideprefix=20&showcount=1',
 'https://en.wikipedia.org/w/index.php?title=Special:CategoryTree&target=Main_topic_classifications&mode=20&hideprefix=20&showcount=1',
 'https://en.wikipedia.org/w/index.php?title=Special:CategoryTree&target=Structure&mode=20&hideprefix=20&showcount=1',
 'https://en.wikipedia.org/wiki/Category:Mathematics',
 'https://en.wikipedia.org/wiki/Category:Fields_of_mathematics',
 'https://en.wikipedia.org/wiki/Category:Algebra',
 'https://en.wikipedia.org/wiki/Category:Mathematical_analysis',
 'https://en.wikipedia.org/wiki/Category:Applied_mathematics']

In [54]:
fout = open('seed_data/test.csv', 'w', newline='')
writer = csv.writer(fout, lineterminator='\n')
for article in articles:
    writer.writerow(article)
fout.close()

In [64]:
with open('seed_data/test.csv', 'w') as out:
    writer = csv.writer(out, lineterminator='\n')
    writer.writerow('')
    for article in articles:
        writer.writerow([''.join(article).strip()])

In [73]:
get_links('mathematics', search_depth=0)

num expand buttons  23
<class 'NoneType'>


Error: iterable expected, not NoneType

In [38]:
link_df.head()

,0
0,<selenium.webdriver.firefox.webelement.Firefox...
1,<selenium.webdriver.firefox.webelement.Firefox...
2,<selenium.webdriver.firefox.webelement.Firefox...
3,<selenium.webdriver.firefox.webelement.Firefox...
4,<selenium.webdriver.firefox.webelement.Firefox...


In [40]:
link_df.to_csv('small_seed_data/test.csv', sep=',', encoding='utf-8', header=None, index=False)

In [ ]:
def save_mongo(items, db_name, collection_name):
    mc = MongoClient()
    db = mc[db_name]
    math = db[collection_name]
    start = timeit.default_timer()
    ref_count = 0
    save_count = 0
    for a in browser.find_elements_by_xpath('.//a'):
        ref_count += 1
        link = a.get_attribute('href')
        cached_article = math.find_one({'aviation': link})
        if cached_article is None:
            save_count += 1
            math.insert_one({'aviation': link})
    end = timeit.default_timer()
    print(str(round((end - start)/60, 2)),  \
          f' minutes to check {ref_count} links and to save {save_count} links.')   

In [ ]:
def save_csv(items):
    elements_list = browser.find_elements_by_xpath('.//a'):

In [4]:
mc = MongoClient()

In [15]:
db = mc['wiki_cache']

In [16]:
type(db)

pymongo.database.Database

In [25]:
av = db.get_collection('aviation')

In [18]:
av

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'wiki_cache'), 'aviation')

In [19]:
hrefs = av.estimated_document_count()

In [20]:
hrefs

49939

In [21]:
href = av.find()

In [22]:
start = 0
end = 1
ref_count = 0
save_count = 0
print(str(round((start-end)/60, 2)),  \
      f' minutes to check {ref_count} links and to save {save_count} links.')

-0.02  minutes to check 0 links and to save 0 links.


In [23]:
count = 0
for ref in href:
    count += 1

In [24]:
count

49939

AttributeError: module 'pandas' has no attribute 'DateFrame'

In [34]:
len(depths)

0

In [39]:
links = pd.DataFrame(depths)

In [42]:
links.to_csv('small_seed_data/MATH_Scrape_D3_31066.csv', sep=',', encoding='utf-8', header=None, index=False)

In [ ]:
\


In [28]:
depth_count = 1
button_count = 0
def expand_all_count_depth(browser):
    """Expand all categories on the page."""
    global depth_count
    global button_count
    expand_buttons = get_expand_buttons(browser)   
    while expand_buttons:
        for button in expand_buttons:
            if button.is_displayed():
                button.click()
                button_count += 1
        print('depth = ', depth_count)
        print('buttons opened = ', button_count)
        depth_count += 1
        expand_buttons = get_expand_buttons(browser)
        

In [29]:
browser = Firefox()
browser.get('https://en.wikipedia.org/wiki/Special:CategoryTree?target=mathematics&mode=all&namespaces=&title=Special%3ACategoryTree')
expand_all_count_depth(browser)

depth =  1
buttons opened =  23
depth =  2
buttons opened =  202
depth =  3
buttons opened =  1251
depth =  4
buttons opened =  4379


In [ ]:
random.randint(1, i + 1)

In [60]:
links = links.loc[12:,:].reset_index()[0]

In [63]:
links.to_csv('small_seed_data/Aviation_Scrape_D_.csv', sep=',', encoding='utf-8', header=None, index=False)

In [61]:
links

0        https://en.wikipedia.org/wiki/Category:Mathema...
1        https://en.wikipedia.org/wiki/Category:Fields_...
2           https://en.wikipedia.org/wiki/Category:Algebra
3        https://en.wikipedia.org/wiki/Category:Algebra...
4        https://en.wikipedia.org/wiki/Category:Theorem...
5        https://en.wikipedia.org/wiki/Category:Abstrac...
6        https://en.wikipedia.org/wiki/Category:Algebra...
7        https://en.wikipedia.org/wiki/Category:Compute...
8        https://en.wikipedia.org/wiki/Category:Element...
9        https://en.wikipedia.org/wiki/Category:History...
10       https://en.wikipedia.org/wiki/Category:Linear_...
11       https://en.wikipedia.org/wiki/Category:Mathema...
12       https://en.wikipedia.org/wiki/Category:Polynom...
13       https://en.wikipedia.org/wiki/Category:Series_...
14       https://en.wikipedia.org/wiki/Category:Symmetr...
15       https://en.wikipedia.org/wiki/Category:Univers...
16       https://en.wikipedia.org/wiki/Category:Variabl.

In [ ]:
for depth in depths:
    print(len(depth))

In [9]:
expand_all_categories(browser)

WebDriverException: Message: Failed to decode response from marionette


In [150]:
for element in driver.find_elements_by_xpath("//span[@data-ct-state='collapsed']"):
    if element.is_displayed():
        element.click()
        for element in driver.find_elements_by_xpath("//span[@data-ct-state='collapsed']"):
            if element.is_displayed():
                element.click()

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=50453): Max retries exceeded with url: /session/e2a9fefa-556b-4cf2-84b8-ad52a64cd185/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f777db4f668>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
print(soup)

In [16]:
print(soup)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category tree - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"CategoryTree","wgNamespaceNumber":-1,"wgPageName":"Special:CategoryTree","wgTitle":"CategoryTree","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":true,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesSho

In [36]:

# Parse into soup() the source of the page after the link is clicked and use "html.parser" as the Parser.
soupify = soup(source, 'html.parser')

In [53]:
soupify.find_all('span',{'class': "CategoryTreeToggle"}).click

[<span class="CategoryTreeToggle CategoryTreeToggleHandlerAttached" data-ct-loaded="1" data-ct-state="expanded" data-ct-title="Mathematics" title="collapse">▼</span>,
 <span class="CategoryTreeToggle CategoryTreeToggleHandlerAttached" data-ct-state="collapsed" data-ct-title="Fields_of_mathematics" title="expand">►</span>,
 <span class="CategoryTreeToggle CategoryTreeToggleHandlerAttached" data-ct-state="collapsed" data-ct-title="Mathematics-related_lists" title="expand">►</span>,
 <span class="CategoryTreeToggle CategoryTreeToggleHandlerAttached" data-ct-state="collapsed" data-ct-title="Mathematicians" title="expand">►</span>,
 <span class="CategoryTreeToggle CategoryTreeToggleHandlerAttached" data-ct-state="collapsed" data-ct-title="Mathematical_concepts" title="expand">►</span>,
 <span class="CategoryTreeToggle CategoryTreeToggleHandlerAttached" data-ct-state="collapsed" data-ct-title="Mathematics_and_culture" title="expand">►</span>,
 <span class="CategoryTreeToggle CategoryTreeTogg

In [31]:


dataList = []
for ultag in soupify.find_all('ul', {'class': 'base-list team-stats'}):
    print(ultag)
    for iltag in ultag.find_all('li'):
        dataList.append(iltag.get_text())

# We are done with the driver so quit.
driver.quit()

In [16]:
print(soup)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category tree - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"CategoryTree","wgNamespaceNumber":-1,"wgPageName":"Special:CategoryTree","wgTitle":"CategoryTree","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":true,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesSho

In [32]:
dataList

[]

In [18]:
url = 'https://en.wikipedia.org/wiki/Special:CategoryTree?target=mathematics&mode=all&namespaces=&title=Special%3ACategoryTree'

response = requests.get(url)
if response.status_code == 200:
    soup = bs(response.text)

print(soup)
    
for a in soup.find_all('a', href=True):
    print("Found the URL:", a['href'])
    
<span class="CategoryTreeToggle" data-ct-state="collapsed" data-ct-title="Philosophy_of_mathematics">

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category tree - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"CategoryTree","wgNamespaceNumber":-1,"wgPageName":"Special:CategoryTree","wgTitle":"CategoryTree","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":false,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgBreakFrames":true,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesSho

In [15]:
def scrape_wiki(csv, url, category):
    """Return the HTML content of url."""
    mc = MongoClient()
    db = mc['cache']
    pages = db[str(category)]
    for i in df:
        url = i
    cached_page = pages.find_one({'url': url})
    if cached_page is None:
        print(f"Fetching {url}")
        response = requests.get(url)
        if response.status_code == 200:
            pages.insert_one({'url': url,
                              'html': response.content})   
        else:
            pages.insert_one({'url': url,
                              'html': None})   
    return pages.find_one({'url': url})['html']

In [16]:
csv = pd.read_csv('data/math/total_math_11697', sep='\t')

In [17]:
csv.loc[0,'url']

'https://en.wikipedia.org/wiki/Alain_Connes'

In [22]:
response = requests.get(csv.loc[0,'url'])

In [33]:
html = response.content
soup = bs(html, 'lxml')

In [37]:
soup.text

'\n\n\nAlain Connes - Wikipedia\ndocument.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );\n(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Alain_Connes","wgTitle":"Alain Connes","wgCurRevisionId":864885442,"wgRevisionId":864885442,"wgArticleId":340,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from June 2013","Articles lacking in-text citations from February 2012","All articles lacking in-text citations","Infobox person using alma mater","Articles with hCards","Wikipedia articles with BIBSYS identifiers","Wikipedia articles with BNF identifiers","Wikipedia articles with GND identifiers","Wikipedia articles with ISNI identifiers","Wikipedia articles with LCCN identifiers","Wikipedia articles with LNB identifiers","Wikipedia artic

In [40]:
#! -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import bleach
import requests


class WikipediaFetcher(object):

    def fetch(self, page_name):
        """
        Passed a Wikipedia page's URL fragment, like
        'Edward_Montagu,_1st_Earl_of_Sandwich', this will fetch the page's
        main contents, tidy the HTML, strip out any elements we don't want
        and return the final HTML string.
        Returns a dict with two elements:
            'success' is either True or, if we couldn't fetch the page, False.
            'content' is the HTML if success==True, or else an error message.
        """
        result = self._get_html(page_name)

        if result['success']:
            result['content'] = self._tidy_html(result['content'])

        return result

    def _get_html(self, page_name):
        """
        Passed the name of a Wikipedia page (eg, 'Samuel_Pepys'), it fetches
        the HTML content (not the entire HTML page) and returns it.
        Returns a dict with two elements:
            'success' is either True or, if we couldn't fetch the page, False.
            'content' is the HTML if success==True, or else an error message.
        """
        error_message = ''

        url = 'https://en.wikipedia.org/wiki/%s' % page_name

        try:
            response = requests.get(url,
                                    params={'action': 'render'}, timeout=5)
        except requests.exceptions.ConnectionError:
            error_message = "Can't connect to domain."
        except requests.exceptions.Timeout:
            error_message = "Connection timed out."
        except requests.exceptions.TooManyRedirects:
            error_message = "Too many redirects."

        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError:
            # 4xx or 5xx errors:
            error_message = "HTTP Error: %s" % response.status_code
        except NameError:
            if error_message == '':
                error_message = "Something unusual went wrong."

        if error_message:
            return {'success': False, 'content': error_message}
        else:
            return {'success': True, 'content': response.text}

    def _tidy_html(self, html):
        """
        Passed the raw Wikipedia HTML, this returns valid HTML, with all
        disallowed elements stripped out.
        """
        html = self._bleach_html(html)
        html = self._strip_html(html)
        return html

    def _bleach_html(self, html):
        """
        Ensures we have valid HTML; no unclosed or mis-nested tags.
        Removes any tags and attributes we don't want to let through.
        Doesn't remove the contents of any disallowed tags.
        Pass it an HTML string, it'll return the bleached HTML string.
        """

        # Pretty much most elements, but no forms or audio/video.
        allowed_tags = [
            'a', 'abbr', 'acronym', 'address', 'area', 'article',
            'b', 'blockquote', 'br',
            'caption', 'cite', 'code', 'col', 'colgroup',
            'dd', 'del', 'dfn', 'div', 'dl', 'dt',
            'em',
            'figcaption', 'figure', 'footer',
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'header', 'hgroup', 'hr',
            'i', 'img', 'ins',
            'kbd',
            'li',
            'map',
            'nav',
            'ol',
            'p', 'pre',
            'q',
            's', 'samp', 'section', 'small', 'span', 'strong', 'sub', 'sup',
            'table', 'tbody', 'td', 'tfoot', 'th', 'thead', 'time', 'tr',
            'ul',
            'var',
            # We allow script here, so we can close/un-mis-nest its tags,
            # but then it's removed completely in _strip_html():
            'script',
        ]

        # These attributes will be removed from any of the allowed tags.
        allowed_attributes = {
            '*':        ['class', 'id'],
            'a':        ['href', 'title'],
            'abbr':     ['title'],
            'acronym':  ['title'],
#           'img':      ['alt', 'src', 'srcset'],
            # Ugh. Don't know why this page doesn't use .tright like others
            # http://127.0.0.1:8000/encyclopedia/5040/
#             'table':    ['align'],
#             'td':       ['colspan', 'rowspan'],
#             'th':       ['colspan', 'rowspan', 'scope'],
        }

        return bleach.clean(html, tags=allowed_tags,
                            attributes=allowed_attributes, strip=True)

    def _strip_html(self, html):
        """
        Takes out any tags, and their contents, that we don't want at all.
        And adds custom classes to existing tags (so we can apply CSS styles
        without having to multiply our CSS).
        Pass it an HTML string, it returns the stripped HTML string.
        """

        # CSS selectors. Strip these and their contents.
        selectors = [
            'div.hatnote',
            'div.navbar.mini',  # Will also match div.mini.navbar
            # Bottom of https://en.wikipedia.org/wiki/Charles_II_of_England :
            'div.topicon',
            'a.mw-headline-anchor',
            'script',
        ]

        # Strip any element that has one of these classes.
        classes = [
            # "This article may be expanded with text translated from..."
            # https://en.wikipedia.org/wiki/Afonso_VI_of_Portugal
            'ambox-notice',
            'magnify',
            # eg audio on https://en.wikipedia.org/wiki/Bagpipes
            'mediaContainer',
            'navbox',
            'noprint',
        ]

        # Any element has a class matching a key, it will have the classes
        # in the value added.
        add_classes = {
            # Give these tables standard Bootstrap styles.
            'infobox':   ['table', 'table-bordered'],
            'ambox':     ['table', 'table-bordered'],
            'wikitable': ['table', 'table-bordered'],
        }

        soup = BeautifulSoup(html, 'html5lib')

        for selector in selectors:
            [tag.decompose() for tag in soup.select(selector)]

        for clss in classes:
            [tag.decompose() for tag in soup.find_all(attrs={'class': clss})]

        for clss, new_classes in add_classes.items():
            for tag in soup.find_all(attrs={'class': clss}):
                tag['class'] = tag.get('class', []) + new_classes

        # Depending on the HTML parser BeautifulSoup used, soup may have
        # surrounding <html><body></body></html> or just <body></body> tags.
        if soup.body:
            soup = soup.body
        elif soup.html:
            soup = soup.html.body

        # Put the content back into a string.
        html = ''.join(str(tag) for tag in soup.contents)

        return html

In [41]:
page = WikipediaFetcher()

In [43]:
page.fetch('Alain_Connes')

{'success': True,
 'content': '<div class="mw-parser-output"><p class="mw-empty-elt">\n</p>\n<table class="plainlinks metadata ambox ambox-style ambox-More_footnotes table table-bordered"><tbody><tr><td class="mbox-image"><div><img alt="" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Text_document_with_red_question_mark.svg/40px-Text_document_with_red_question_mark.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Text_document_with_red_question_mark.svg/60px-Text_document_with_red_question_mark.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Text_document_with_red_question_mark.svg/80px-Text_document_with_red_question_mark.svg.png 2x"/></div></td><td class="mbox-text"><div class="mbox-text-span">This article includes a <a href="//en.wikipedia.org/wiki/Wikipedia:Citing_sources" title="Wikipedia:Citing sources">list of references</a>, but <b>its sources remain unclear</b> because it has <b>insufficient <a href="//en.wikipedia.org/wiki/Wiki